<a href="https://colab.research.google.com/github/flatneuron/football-player-reidentification/blob/main/data_collection_football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
# Load a custom trained model with the corrected path
from ultralytics import YOLO
model = YOLO("drive/MyDrive/stealth mode/model.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load model (with your custom weights)
model = YOLO("drive/MyDrive/stealth mode/model.pt")

# Video input/output path
input_path = "drive/MyDrive/stealth mode/soccer_match.mp4"
output_path = "drive/MyDrive/stealth mode/29_june_bytetrack_output_video.mp4"


In [ ]:
import cv2

# Define colors for each class
CLASS_COLORS = {
    "player": (0, 0, 0),      # black
    "referee": (255, 0, 0),     # Blue
    "ball": (0, 0, 255)         # Red
}

def draw_detections(
    frame,
    detections,
    class_names,
    show_tracking=False
):
    """
    Args:
        frame (np.ndarray): The image frame (BGR)
        detections (list of dict): Each dict contains:
            - 'box': [x1, y1, x2, y2]
            - 'conf': float
            - 'cls': int (index in class_names)
            - 'id': int (optional, if show_tracking is True)
        class_names (dict): {0: 'ball', 1: 'referee', 2: 'player'}
        show_tracking (bool): Whether to draw track IDs
    """
    for det in detections:
        box = list(map(int, det['box']))
        conf = det['conf']
        cls_id = det['cls']
        track_id = det.get('id', None)

        label = f"{class_names[cls_id]} {conf:.2f}"
        if show_tracking and track_id is not None:
            label += f" ID:{track_id}"

        color = CLASS_COLORS.get(class_names[cls_id], (255, 255, 255))

        # Draw bbox
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color, 2)

        # Measure text size
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        thickness = 1
        (text_w, text_h), baseline = cv2.getTextSize(label, font, font_scale, thickness)

        # Text box outline (no fill)
        top_left = (box[0], box[1] - text_h - 6)
        bottom_right = (box[0] + text_w + 6, box[1])
        cv2.rectangle(frame, top_left, bottom_right, color, 1)

        # Put label text
        cv2.putText(frame, label, (box[0] + 3, box[1] - 3), font, font_scale, color, thickness)

    return frame


In [ ]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
import cv2
from ultralytics import YOLO
from supervision import ByteTrack, Detections

# === CONFIG ===
VIDEO_PATH = "drive/MyDrive/stealth mode/soccer_match.mp4"
MODEL_PATH = "drive/MyDrive/stealth mode/model.pt"
OUTPUT_VIDEO_PATH = "drive/MyDrive/stealth mode/new_output_with_tracks.mp4"
CROP_SAVE_DIR = "drive/MyDrive/stealth mode/new_trackid_crops"
CONF_THRESH = 0.4
MAX_SECONDS = 5

# === CLASS COLORS AND DRAW FUNCTION ===
CLASS_COLORS = {
    "player": (0, 0, 0),      # black
    "referee": (255, 0, 0),   # blue
    "ball": (0, 0, 255)       # red
}

def draw_detections(frame, detections, class_names, show_tracking=False):
    for det in detections:
        box = list(map(int, det['box']))
        conf = det['conf']
        cls_id = det['cls']
        track_id = det.get('id', None)

        label = f"{class_names[cls_id]} {conf:.2f}"
        if show_tracking and track_id is not None:
            label += f" ID:{track_id}"

        color = CLASS_COLORS.get(class_names[cls_id], (255, 255, 255))

        # Draw bbox
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color, 2)

        # Measure text size
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        thickness = 1
        (text_w, text_h), baseline = cv2.getTextSize(label, font, font_scale, thickness)

        # Text box outline (no fill)
        top_left = (box[0], box[1] - text_h - 6)
        bottom_right = (box[0] + text_w + 6, box[1])
        cv2.rectangle(frame, top_left, bottom_right, color, 1)

        # Put label text
        cv2.putText(frame, label, (box[0] + 3, box[1] - 3), font, font_scale, color, thickness)

    return frame

# === INITIALIZATION ===
os.makedirs(CROP_SAVE_DIR, exist_ok=True)
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
max_frames = int(fps * MAX_SECONDS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

byte_tracker = ByteTrack()
frame_idx = 0
class_names = model.names  # e.g., {0: 'ball', 1: 'referee', 2: 'player'}

# === FRAME-BY-FRAME PROCESSING ===
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    result = model(frame)[0]
    boxes = result.boxes

    if boxes is not None and len(boxes) > 0:
        det = Detections(
            xyxy=boxes.xyxy.cpu().numpy(),
            confidence=boxes.conf.cpu().numpy(),
            class_id=boxes.cls.cpu().numpy().astype(int)
        )

        det = det[det.confidence > CONF_THRESH]

        tracks = byte_tracker.update_with_detections(det)

        detections_to_draw = []

        for i in range(len(tracks)):
            track_id = int(tracks.tracker_id[i])
            class_id = int(tracks.class_id[i])
            x1, y1, x2, y2 = map(int, tracks.xyxy[i])
            conf = float(tracks.confidence[i])

            detections_to_draw.append({
                "box": [x1, y1, x2, y2],
                "conf": conf,
                "cls": class_id,
                "id": track_id
            })

            # Save crops in first few seconds
            if frame_idx < max_frames:
                crop = frame[y1:y2, x1:x2]
                if crop.size != 0:
                    save_path = os.path.join(
                        CROP_SAVE_DIR,
                        f"{frame_idx:04d}_{track_id:04d}.jpg"
                    )
                    cv2.imwrite(save_path, crop)

        # Draw detections using your custom function
        frame = draw_detections(
            frame=frame,
            detections=detections_to_draw,
            class_names=class_names,
            show_tracking=True
        )

    out.write(frame)
    frame_idx += 1

# === CLEANUP ===
cap.release()
out.release()
print(f"✅ Done: saved crops from first {MAX_SECONDS} seconds and output video.")



0: 384x640 1 ball, 16 players, 2 referees, 3540.8ms
Speed: 19.9ms preprocess, 3540.8ms inference, 55.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2831.7ms
Speed: 4.5ms preprocess, 2831.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2935.9ms
Speed: 4.0ms preprocess, 2935.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2872.2ms
Speed: 3.8ms preprocess, 2872.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3467.4ms
Speed: 5.2ms preprocess, 3467.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2819.4ms
Speed: 2.9ms preprocess, 2819.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2916.3ms
Speed: 3.7ms preprocess, 2916.3ms inference, 1.5m